<a href="https://colab.research.google.com/github/SahilCarterr/FinetuningLLM/blob/main/distill_bert_nueral_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('/content/data_withoutLENE.csv')
unique_topics = df['topic'].unique()
topic_mapping = {topic: i for i, topic in enumerate(unique_topics)}

# Apply the mapping to the 'topic' column
df['topic'] = df['topic'].map(topic_mapping)

# Display the updated DataFrame and the mapping of unique strings to numbers
print("Mapping of unique strings to numbers:", topic_mapping)
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test datasets to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Data has been split and saved to train.csv and test.csv.")


Mapping of unique strings to numbers: {'Dating and Relationships': 0, 'Random': 1, 'Movies and TV': 2, 'Music': 3, 'Technology and Internet': 4, 'Philosophy and Religion': 5, 'technology and Internet': 6, 'Law and Order': 7, 'Health and Wellness': 8, 'Food and Drinks': 9, 'Fashion and Makeup': 10, 'health and Wellness': 11, 'Gaming': 12, 'Sports': 13, 'fashion and Makeup': 14, 'sports': 15, 'Anime': 16, 'anime': 17, 'Finance': 18, 'Travel and Geography': 19, 'Pets and Animals': 20, 'gaming': 21, 'random': 22, 'music': 23}
Data has been split and saved to train.csv and test.csv.


In [2]:
!pip -q install pyarrow datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [1]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'topic'],
        num_rows: 1274
    })
    test: Dataset({
        features: ['question', 'topic'],
        num_rows: 319
    })
})

In [16]:
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    # Ensure inputs are strings, handle potential None or other non-string types
    questions = [str(q) if q is not None else "" for q in examples['question']]
    return tokenizer(questions,truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets = tokenized_datasets.remove_columns(["question"])
tokenized_datasets = tokenized_datasets.rename_column("topic", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [18]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [19]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [4]:

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [20]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [21]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 24)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [10]:
model = DistillBERTClass()
model.to(device)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [22]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [23]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [27]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(train_dataloader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%10==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [44]:
EPOCHS=2
for epoch in range(EPOCHS):
    train(epoch)

Training Loss per 5000 steps: 0.09685271233320236
Training Accuracy per 5000 steps: 100.0
Training Loss per 5000 steps: 0.2311187372966246
Training Accuracy per 5000 steps: 93.18181818181819
Training Loss per 5000 steps: 0.2129516461420627
Training Accuracy per 5000 steps: 95.23809523809524
Training Loss per 5000 steps: 0.20738010517051142
Training Accuracy per 5000 steps: 95.96774193548387
Training Loss per 5000 steps: 0.22861069513530266
Training Accuracy per 5000 steps: 95.42682926829268
Training Loss per 5000 steps: 0.2180613014189636
Training Accuracy per 5000 steps: 95.58823529411765
Training Loss per 5000 steps: 0.22141745997989765
Training Accuracy per 5000 steps: 95.69672131147541
Training Loss per 5000 steps: 0.2141282775137626
Training Accuracy per 5000 steps: 95.95070422535211
Training Loss per 5000 steps: 0.1995830290755372
Training Accuracy per 5000 steps: 96.45061728395062
Training Loss per 5000 steps: 0.1953718384707367
Training Accuracy per 5000 steps: 96.7032967032967

In [47]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    tr_loss = 0  # Initialize tr_loss here
    nb_tr_steps = 0
    nb_tr_examples = 0 # Initialize these variables as well
    with torch.no_grad():
        for _, data in enumerate(eval_dataloader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            targets = data['labels'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%10==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 10 steps: {loss_step}")
                print(f"Validation Accuracy per 10 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [48]:
acc = valid(model, eval_dataloader)
print("Accuracy on test data = %0.2f%%" % acc)

Validation Loss per 10 steps: 2.7464756965637207
Validation Accuracy per 10 steps: 37.5
Validation Loss per 10 steps: 3.1711302453821357
Validation Accuracy per 10 steps: 37.5
Validation Loss per 10 steps: 3.1558673608870733
Validation Accuracy per 10 steps: 34.523809523809526
Validation Loss per 10 steps: 3.2177024425998813
Validation Accuracy per 10 steps: 34.67741935483871
Validation Loss Epoch: 3.0272099390625953
Validation Accuracy Epoch: 37.30407523510972
Accuracy on test data = 37.30%
